In [121]:
from requests import get
from bs4 import BeautifulSoup
from lxml import etree
from datetime import datetime, date, timedelta

In [122]:
import pandas as pd
import numpy as np

import time
import re

In [181]:
# get search url #https://medium.com/data-hackers/tutorial-para-iniciantes-web-scraping-em-python-bb71778ed40c
url = 'https://safras.com.br/eng/agribusiness/'
response = get(url)
# create soup
soup = BeautifulSoup(response.text, 'html.parser')
# get Dom
dom = etree.HTML(str(soup))

In [182]:
# get last news
title = dom.xpath('//*[@id="tdi_39"]/div[1]/div/div[2]/h3/a')[0].get('title')
link = dom.xpath('//*[@id="tdi_39"]/div[1]/div/div[2]/h3/a')[0].get('href')

# get data author and date
author = dom.xpath('//*[@id="tdi_39"]/div[1]/div/div[2]/div[1]/span/span[1]/a')[0].text
date = dom.xpath('//*[@id="tdi_39"]/div[1]/div/div[2]/div[1]/span/span[2]/time')[0].get('datetime')[:10]
last_news = 'true'

In [183]:
# validate date
today = datetime.today()
yesterday = (today - timedelta(days=1)).strftime('%Y-%m-%d')
week_ago = today - timedelta(days=7)

# Validate
yesterday2 = (today - timedelta(days=3)).strftime('%Y-%m-%d')
yesterday2

'2024-01-23'

In [184]:
# create frame last news
frame = pd.DataFrame(columns=['data_process','title','link','author','date_publish','last_news'], dtype=str)

# set schema
schema = {
    'data_process':today.strftime('%Y-%m-%d'),
    'title':title,
    'link':link,
    'author':author,
    'date_publish':date,
    'last_news': last_news
    }

frame = pd.concat([frame, pd.DataFrame([schema])])

In [185]:
frame

,data_process,title,link,author,date_publish,last_news
0,2024-01-26,"Early harvest in Brazil, USDA, and supply size...",https://safras.com.br/eng/early-harvest-in-bra...,Ritiele Rodrigues,2024-01-23,true


In [186]:
# Count of array
count_array = len(soup.findAll(class_='td-module-meta-info'))

# Loop for get data
for count_ in range(1, count_array):
    title = soup.findAll(class_='td-module-meta-info')[count_].h3.a['title']
    link = soup.findAll(class_='td-module-meta-info')[count_].h3.a['href']
    date = soup.findAll(class_='td-module-meta-info')[count_].div.span.span.time['datetime'][:10]
    last_news = 'false'
    
    schema = {
        'data_process':today.strftime('%Y-%m-%d'),
        'title':title,
        'link':link,
        'author':'null',
        'date_publish':date,
        'last_news': last_news
        }
    
    frame = pd.concat([frame, pd.DataFrame([schema])])

In [187]:
frame['send_mail'] = frame['date_publish'].map(lambda x: 'true' if x == yesterday else 'false')
frame['send_mail2'] = frame['date_publish'].map(lambda x: 'true' if x == yesterday2 else 'false')

In [188]:
frame

,data_process,title,link,author,date_publish,last_news,send_mail,send_mail2
0,2024-01-26,"Early harvest in Brazil, USDA, and supply size...",https://safras.com.br/eng/early-harvest-in-bra...,Ritiele Rodrigues,2024-01-23,true,false,true
0,2024-01-26,"With a smaller soybean crop, Brazil is forecas...",https://safras.com.br/eng/with-a-smaller-soybe...,null,2024-01-16,false,false,false
0,2024-01-26,Exchanges increase potential production of cor...,https://safras.com.br/eng/exchanges-increase-p...,null,2024-01-16,false,false,false
0,2024-01-26,The live pig x corn exchange ratio worsened in...,https://safras.com.br/eng/the-live-pig-x-corn-...,null,2024-01-11,false,false,false
0,2024-01-26,"With growers revising the soybean crop size, 2...",https://safras.com.br/eng/with-growers-revisin...,null,2024-01-09,false,false,false
